In [7]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [8]:
# Load train.csv containing file paths and labels
df_train = pd.read_csv("../CSV Files/train.csv")
df_test = pd.read_csv("../CSV Files/test.csv")

In [9]:
def augment_frame(frame):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2]
    )
    frame = np.expand_dims(frame, axis=0)
    frame = datagen.flow(frame, batch_size=1)[0][0]
    return frame

In [10]:
def extract_frames(video_path, num_frames):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // num_frames)
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (299, 299))
        frame = img_to_array(frame) / 255.0
        # frame = augment_frame(frame)  # Apply data augmentation
        frames.append(frame)
    
    cap.release()
    return np.array(frames)

In [11]:
def prepare_dataset(df, num_frames=20):
    X, y = [], []
    for index, row in df.iterrows():
        video_path, label = row['Video Name'], row['Tag']
        frames = extract_frames(video_path, num_frames)
        if frames.shape[0] == num_frames:
            X.append(frames)
            y.append(1 if label.lower() == "fake" else 0)  # Convert 'fake' -> 1, 'real' -> 0
    return np.array(X), np.array(y, dtype=np.float32)  # Ensure numerical dtype


In [12]:
# Prepare training and testing data
X_train, y_train = prepare_dataset(df_train)
X_test, y_test = prepare_dataset(df_test)

In [17]:
# Define Model Architecture
input_layer = Input(shape=(20, 299, 299, 3)) 

In [18]:
# Xception base model for feature extraction
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.trainable = True  # Enable fine-tuning

# Unfreeze last 10 layers for fine-tuning
for layer in base_model.layers[-5:]:
    layer.trainable = True

# TimeDistributed wrapper to apply Xception on each frame
x = TimeDistributed(base_model)(input_layer)
x = TimeDistributed(GlobalAveragePooling2D())(x)

# LSTM layers for sequential analysis
x = LSTM(128, return_sequences=True)(x)
x = LSTM(64, return_sequences=False)(x)

# Fully connected layers
x = Dense(64, activation='relu')(x)
out = Dense(1, activation='sigmoid')(x)

In [19]:
# Build and compile the model
model = Model(inputs=input_layer, outputs=out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Train model
model.fit(X_train, y_train, epochs=6, batch_size=4, validation_split=0.2)

Epoch 1/6


: 

In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


In [ ]:
# Save the model
model.save("../Models/Xception_LSTM/xception_lstm2.keras")
print("Model saved as xception_lstm.keras")